In [10]:
from sqlalchemy import create_engine, URL, inspect
import pandas as pd

from dotenv import load_dotenv
import os


def load_data():
    # df1 = bachelors_degree()
    # df2 = median_income()
    # df3 = crime_rate()
    # df4 = poverty_rate()
    # df5 = unemployment_rate()
    # df6 = home_value()

    load_dotenv()

    db_url = URL.create(
        "mysql+pymysql",
        username=os.getenv("MYSQL_USER"),
        password=os.getenv("MYSQL_PASSWORD"),
        host=os.getenv("MYSQL_HOST"),
        database=os.getenv("MYSQL_DATABASE")
    )
    engine = create_engine(db_url)

    # df1.to_sql('bachelors_degree', engine, index=False)
    # df2.to_sql('median_income', engine, index=False)
    # df3.to_sql('crime_rate', engine, index=False)
    # df4.to_sql('poverty_rate', engine, index=False)
    # df5.to_sql('unemployment_rate', engine, index=False)
    # df6.to_sql('home_value', engine, index=False)

    print(inspect(engine).get_table_names())


def bachelors_degree():
    df1 = pd.read_csv("../data/bachelors_degree_by_state.csv")
    df2 = pd.read_csv("../predicted_data/predicted_vs_actual_bachelors_degrees_2022.csv")
    df3 = pd.read_csv("../predicted_data/predicted_bachelors_degrees_2023.csv")
    df4 = pd.read_csv("../predicted_data/predicted_bachelors_degree_by_state_2024.csv")

    df2 = df2.iloc[:, 0:2]
    df4 = df4.rename(columns={'Predicted 2024 Percentage': 'Predicted 2024'})

    df1 = df1.join(df2.set_index('State'), on='State')
    df1 = df1.join(df3.set_index('State'), on='State')

    return df1.join(df4.set_index('State'), on='State')


def median_income():
    df1 = pd.read_csv("../data/median_household_income.csv")
    df2 = pd.read_csv("../predicted_data/median_income_predictions_2022_2023_2024.csv")

    df2 = df2.rename(columns={'Predicted 2022 Median Income': 'Predicted 2022',
                              'Predicted 2023 Median Income': 'Predicted 2023',
                              'Predicted 2024 Median Income': 'Predicted 2024'})
    df2.pop('Actual 2022 Median Income')
    df2.insert(len(df2.columns) - 1, 'Error 2022', df2.pop('Error 2022'))

    median_income_df = df1.join(df2.set_index('State'), on='State')

    return median_income_df


def crime_rate():
    df1 = pd.read_csv("../data/State-data.csv")
    df2 = pd.read_csv("../predicted_data/crime_poverty_predictions_2022_2023_2024.csv")

    df1 = df1.pivot_table(index='State', columns='Year', values='Violent Crime per hundred thousand people')
    df1 = df1.reset_index()
    df1.columns.name = ''

    df2 = df2.iloc[:, 0:4]
    df2 = df2.rename(columns={'Predicted 2022_Crime': 'Predicted 2022',
                              'Predicted 2023_Crime': 'Predicted 2023',
                              'Predicted 2024_Crime': 'Predicted 2024'})

    crime_rate_df = df1.join(df2.set_index('State'), on='State')

    return crime_rate_df


def poverty_rate():
    df1 = pd.read_csv("../data/State-data.csv")
    df2 = pd.read_csv("../predicted_data/crime_poverty_predictions_2022_2023_2024.csv")

    df1 = df1.pivot_table(index='State', columns='Year', values='Poverty in thousands')
    df1 = df1.reset_index()
    df1.columns.name = ''

    states = df2.iloc[:, 0:1]
    df2 = df2.iloc[:, 4:]
    df2 = pd.concat([states, df2], axis=1)

    poverty_rate_df = df1.join(df2.set_index('State'), on='State')

    return poverty_rate_df


def unemployment_rate():
    df1 = pd.read_csv("../data/Unemployment in America Per US State.csv")

    df1.drop(df1.iloc[:, 0:1], axis=1, inplace=True)
    df1.drop(df1.iloc[:, 3:9], axis=1, inplace=True)
    df1 = df1.rename(columns={"Percent (%) of Labor Force Unemployed in State/Area": "Percent Unemployed"})
    df1 = df1.groupby(['State/Area', 'Year']).mean()
    df1.pop('Month')

    df1 = df1.pivot_table(index='State/Area', columns='Year', values='Percent Unemployed')
    df1 = df1.reset_index()
    df1.columns.name = ''

    df1 = df1.drop([18, 34])
    df1 = df1.reset_index()
    df1.pop('index')

    return df1


def home_value():
    df = pd.read_csv("../data/ZHVI_bystate_monthly.csv")
    df.drop(df.iloc[:, 0:2], axis=1, inplace=True)
    df.drop(df.iloc[:, 1:3], axis=1, inplace=True)

    states = df.iloc[:, 0:1]
    states = states.rename(columns={'RegionName': 'State'})
    df.pop('RegionName')
    df.columns = pd.to_datetime(df.columns)
    df = df.groupby(df.columns.year, axis=1).mean()

    return states.join(df)


if __name__ == '__main__':
    load_data()


ImportError: cannot import name 'URL' from 'sqlalchemy' (/Users/siyuhan/anaconda3/lib/python3.11/site-packages/sqlalchemy/__init__.py)

In [7]:
!pip install sqlalchemy

In [8]:
!pip install dotenv

  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [83 lines of output]
      /Users/siyuhan/anaconda3/lib/python3.11/site-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [17 lines of output]
            Traceback (most recent call last):
              Fi

In [1]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
degree = 2
polyreg_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
polyreg_model.fit(X_train, y_train)
predictions = polyreg_model.predict(X_predict)


NameError: name 'X_train' is not defined

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Load the data
data = pd.read_csv('Yearly_Unemployment_Rate_by_State.csv')

# Assume the structure has columns 'Year' and 'UnemploymentRate'
X = data['Year'].values.reshape(-1, 1)  # Reshape for sklearn
y = data['UnemploymentRate'].values

# Setup linear regression model
linear_model = LinearRegression()

# Fit the model
linear_model.fit(X, y)

# Predict for future years
years_to_predict = np.array([[2023], [2024]])
predictions = linear_model.predict(years_to_predict)

# Print the predictions
print("Predicted Unemployment Rates:")
print("2023:", predictions[0])
print("2024:", predictions[1])


KeyError: 'UnemploymentRate'

In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Load the data
data = pd.read_csv('Yearly_Unemployment_Rate_by_State.csv')

# Pivot the data to have years as rows and states as columns
data_pivot = data.pivot(index='Year', columns='State', values='Average Unemployment Rate (%)')

# Prepare to predict for each state
states = data_pivot.columns
years_to_predict = np.array([[2023], [2024]])

# Dictionary to store predictions for each state
state_predictions = {}

# Loop through each state to create and train a model
for state in states:
    X = data_pivot.index.values.reshape(-1, 1)  # Years as the independent variable
    y = data_pivot[state].values  # Unemployment rates as the dependent variable
    
    # Setup and train linear regression model
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict for 2023 and 2024
    predictions = model.predict(years_to_predict)
    state_predictions[state] = predictions

# Display the predicted unemployment rates for each state
print("Predicted Unemployment Rates for 2023 and 2024:")
for state, preds in state_predictions.items():
    print(f"{state} - 2023: {preds[0]:.2f}%, 2024: {preds[1]:.2f}%")


Predicted Unemployment Rates for 2023 and 2024:
Alabama - 2023: 4.53%, 2024: 4.44%
Alaska - 2023: 5.72%, 2024: 5.64%
Arizona - 2023: 5.62%, 2024: 5.59%
Arkansas - 2023: 4.43%, 2024: 4.36%
California - 2023: 6.83%, 2024: 6.82%
Colorado - 2023: 4.55%, 2024: 4.52%
Connecticut - 2023: 5.63%, 2024: 5.64%
Delaware - 2023: 4.61%, 2024: 4.58%
District of Columbia - 2023: 6.83%, 2024: 6.80%
Florida - 2023: 4.96%, 2024: 4.92%
Georgia - 2023: 5.64%, 2024: 5.63%
Hawaii - 2023: 4.12%, 2024: 4.09%
Idaho - 2023: 4.06%, 2024: 3.99%
Illinois - 2023: 6.07%, 2024: 6.04%
Indiana - 2023: 4.62%, 2024: 4.57%
Iowa - 2023: 3.45%, 2024: 3.40%
Kansas - 2023: 4.58%, 2024: 4.58%
Kentucky - 2023: 5.49%, 2024: 5.45%
Los Angeles County - 2023: 7.61%, 2024: 7.62%
Louisiana - 2023: 4.95%, 2024: 4.87%
Maine - 2023: 4.20%, 2024: 4.14%
Maryland - 2023: 4.68%, 2024: 4.66%
Massachusetts - 2023: 4.91%, 2024: 4.89%
Michigan - 2023: 5.63%, 2024: 5.54%
Minnesota - 2023: 4.04%, 2024: 4.01%
Mississippi - 2023: 6.06%, 2024: 6.01%
